In [1]:
import pandas as pd
import ast

import plotly # only necessary for plotting
from plotly import tools # only necessary for plotting
import plotly.graph_objs as go # only necessary for plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot # only necessary for plotting
init_notebook_mode(connected=True) # only necessary for plotting

In [9]:
# reading PV file - if PV is in Navid's format (including 'â†µ' for new line)
flag = True
filename = "LP_testing_PV"

if flag == True:
    text_file = open(filename + ".txt", "r")
    x = text_file.readlines()
    x[0] = x[0].replace('â†µ', ' ')

    span = 2
    x2 = x[0].split(" ")
    x3 = [" ".join(x2[i:i+span]) for i in range(0, len(x2), span)]
    x4 = pd.DataFrame([sub.split(",") for sub in x3])

    PVOutput = x4.copy()
    PVOutput.columns = ['TS', 'PV']
    PVOutput.to_csv(filename + ".csv",index=False)
    
else:
    # reading PV file - if PV is in Phoebe's format (including \n for new line)
    PVOutput=pd.read_csv(filename + '.txt',header=None)
    PVOutput.columns = ['TS', 'PV']
    PVOutput.to_csv(filename + ".csv",index=False)

In [17]:
# reading load file
# this will throw an error if the text file has null instead of 0, the simplest fix is to do a find and replace in the txt file
filename = "LP_Mosman_allgas"
load_text_file = open(filename + ".txt", "r")
x = load_text_file.readlines()
b = ast.literal_eval(x[0])
x2 = pd.DataFrame(b)
x2.head()
if x2.shape[1] == 2:
    x2.columns = ['TS_Epoch', 'kWh']
elif x2.shape[1] == 3:
    x2.columns = ['TS_Epoch', 'kWh', 'kW']
elif x2.shape[1] == 4:
    x2.columns = ['TS_Epoch', 'kWh', 'kW', 'TOU']
elif x2.shape[1] == 9:
    x2.columns = ['TS_Epoch', 'PV', 'load', 'SOC', 'excess_PV', 'excess_load', 'TOU', 'excess_charge', 'NetLoad']

x2['TS'] = pd.to_datetime(x2['TS_Epoch'], unit='ms')

LoadOutput = x2[['TS','kWh']].copy()
#LoadOutput = x2.copy()
LoadOutput.to_csv(filename + ".csv",index=False)

In [18]:
LoadProf = pd.read_csv("LP_Mosman_allgas.csv")
python_load = pd.read_csv("LP_Mosman_nogas.csv")
#python_load2 = pd.read_csv("LP_Mosman1_python2_processed.csv")
#python_load3 = pd.read_csv("LP_Mosman1_2_Processed.csv")

LoadProf['TS']=pd.to_datetime(LoadProf['TS'], infer_datetime_format=True)
python_load['TS']=pd.to_datetime(python_load['TS'], infer_datetime_format=True)
#python_load2['TS']=pd.to_datetime(python_load2['TS'], infer_datetime_format=True)
#python_load3['TS']=pd.to_datetime(python_load3['TS'], infer_datetime_format=True)

fig = go.Figure()

fig.add_trace(go.Scatter(x= LoadProf['TS'], y= LoadProf['kWh'],name='SunSPoT Load profile (kWh)', mode='lines'))

fig.add_trace(go.Scatter(x= python_load['TS'], y= python_load['kWh'],name='Python Load profile (kWh)', mode='lines'))

#fig.add_trace(go.Scatter(x= python_load2['TS'], y= python_load2['kWh'],name='Python Load profile 2 (kWh)', mode='lines'))

#fig.add_trace(go.Scatter(x= python_load3['TS'], y= python_load3['kWh'],name='Python Load profile 3 (kWh)', mode='lines'))

fig.update_layout(title='Load profile' ,xaxis=dict(title='Time'),yaxis=dict(title='Power (kWh)'))

iplot(fig)